# RefAssig V0.0

this is my faster simple version of the PMI and Abstract gene count scoring system.
Improvements: 
* rapid access
* streamlined function calls
* clean data output
* xml abstract data parsing

---
## 1.0 Libraries and input

The work flow looks like this:

1. read in a list of chemicals as a csv with a column named chemicals and all chemicals examined in this study
2. read in a query map as a csv
    * a query map is a relationship map between all query phrases and all classifiers
    * three columns are used:
        * A general class/super class for a query
        * a direction: 'pos' or 'neg' refereing to positive or negative
        * the search phrase surrounded by quotes, e.g., ' "dna damage" '
        
The work flow will automatically construction all wanted querries between chemicals and search phrases from these list. In later steps the results can be reduced to a meaningfull level by knowing if search phrases are addative for the target or subtractive e.g., metal stresss vs chelator. Here, a high number of chelator hits and metal stress hits might indicate that although the chemical might have a significant response for metal stress it also is likely related tagentially to the overall function. Think about what queries you are supplying to ensure they are meaningful.

---
# RAPID QUERY FUNCTION

In [33]:
#This function reads csv with a list of chemical names and a query map and returns a list of dataframes for all combinations of queries in the data set
#this result can be input to two functions: ```PMI_matrix``` which will give you information about the information or strength of the search phrase relative to the total 

query_result = function(chemical.list.path, query.map.path = NULL, limit_input_to = NULL, output.path) {

  

    #Libraries
    suppressMessages(library(tidyverse))
    suppressMessages(library(easyPubMed))
    suppressMessages(library(kableExtra))
    suppressMessages(library(parallel))

    #testingpaths
    chemical.list.path = "input/chemical-list-LINCS-2021.07.23.csv"
    query.map.path = "input/query.map-build.csv"
    #query.map.path = NULL
    limit_input_to = NULL

if(!is.null(query.map.path)) { 
    ######################
    #SEARCH WITH QUERY MAP
    
    #read chemical list
    chems = suppressMessages(read_csv(paste0(chemical.list.path)))
    #read query map
    q.map = suppressMessages(read_csv(paste0(query.map.path))) %>%
        mutate(phrase.q = paste0('"',phrase,'"'))
    
    #cutchems
    if(is.null(limit_input_to)) {chems = chems$chemicals[1:length(chems$chemicals)]} else {chems = chems$chemicals[1:limit_input_to]}

    #build partial search vectors
    chems.search = paste0('"', chems,'" AND ')
    searchterms.search = q.map$phrase.q

    #controlable and bindable search grid with directions
    search.combinations = expand.grid(chems.search, searchterms.search, stringsAsFactors = FALSE) %>%
    setNames(c('chemical', 'phrase.q')) %>% 
    left_join(q.map, by = "phrase.q") %>%
    mutate(query = paste(chemical, phrase.q)) %>%
    select(SRP, direction, query)

    #acutal query list
    search.query = search.combinations$query %>% as.list()
    
    } else {
    #######################    
    #SEARCH WITH ONLY CHEMS
        
    #read chemical list      
    chems = suppressMessages(read_csv(paste0(chemical.list.path)))
    
    #cutchems
    if(is.null(limit_input_to)) {chems = chems$chemicals[1:length(chems$chemicals)]} else {chems = chems$chemicals[1:limit_input_to]}

    #build partial search vectors
    chems.search = paste0('"', chems,'"')
        
    #actual query list
    search.query = chems.search
    }

    #build output object
    query.result = vector(length = 3, mode = "list")

    #query
    query.result[[1]] = mclapply(seq_along(search.query), function(x){
        
        Sys.sleep(time = 2)     
        
        get_pubmed_ids(search.query[[x]], api_key = 'b3accc43abfb376d63b0c2d2f7d8f984de09') %>%
           {if(class(.)  == 'try-error') 'TRY-FAILED-increase sys sleep' else
               {if(as.numeric(.$Count) > 0) fetch_pubmed_data(., retstart = 0, retmax = 100000) %>%
                                            articles_to_list() %>%
                                            lapply(., article_to_df, max_chars = 100000, getAuthors = FALSE) %>%
                                            do.call(rbind, .) else 'EMPTY' }}
        
            }, mc.cores = 10) %>% setNames(search.query)
  

    #clean out empty hits
    query.result[[1]] = query.result[[1]][query.result[[1]] != 'EMPTY']

    #save additional mapping information if a map was used
    if(is.null(query.map.path)) {query.result[[2]] = 'NO MAP USED'} else {query.result[[2]] = search.combinations}
    if(is.null(query.map.path)) {query.result[[3]] = 'NO MAP USED'} else {query.result[[3]] = q.map}

    
    #save result
    base::save(query.result, file = paste0("../AbstractR-projects/LINCS/output/query.result_LINCS_", Sys.Date(),".RData"))

    

    return(query.result)

   
}

---
# ABSTRACT GENE COUNT PARSING FINAL FUNCTION

[ ] update prioritization of chemical over abstract for gene parsing line 54 - change outter apply to MC and inner to lapply

'''#isolate genes from all abstracts
    abstracts.genes.filtered = lapply(seq_along(abstracts), function(y)
        mclapply(1:nrow(abstracts[[y]]), function(x) abstracts[[y]][x,2] %>%
            remove.punc() %>% match.genes(mouse.rat.vec), mc.cores = 32) %>% unlist()) %>% setNames(names(abstracts))`''

In [10]:
#This function reads csv with a list of chemical names and a query map and returns a list of dataframes for all combinations of queries in the data set
#this result can be input to two functions: ```PMI_matrix``` which will give you information about the information or strength of the search phrase relative to the total 

abstract_gene_counts = function(query_result_object) {    

    #Libraries
    suppressMessages(library(tidyverse))
    library(parallel)

    #helper functions
    remove.punc = function(x) {
        string.1 = gsub("(?!\\.)[[:punct:]]", "", x, perl=TRUE)
        string.2 = substr(string.1,1,nchar(string.1)-1) #%>% tolower()
        string.3 = str_split(string.2, " ") %>% unlist()
    return(string.3)
    }

    match.genes <- function(sentence, genelist) {
    gene.matches = sentence[sentence %in% genelist]
    return(gene.matches)
    }

    #testingpaths
    #query_result_object = "../AbstractR-projects/LINCS/output/query.result_LINCS_2021-07-29.RData"
    
    #read query result
    load(paste0(query_result_object))

    #set up genes dictionary - improve this in the future, allow for a list of homolouges in a csv from bioMart - for now use your H-M-R set up
    #read in coding genes
    #also a data dir or lib is needed
    gene.list = suppressMessages(suppressWarnings(read_tsv("data/proteincodinggenesHGNC.txt"))) %>%
    pull(symbol) %>% unique()
    
    #read in non human gene lists homolouge lists
    nh.gene.list <- suppressMessages(read_csv("data/genes.list_h.m.r.csv")) %>% filter(human %in% gene.list) %>% distinct(`.keep_all` = FALSE)
    
    #comparative lists
    mouse.rat.vec = c(nh.gene.list$human, nh.gene.list$mouse, nh.gene.list$rat) %>% unique()

    #filter some bad gene names
    bad.genes.q = c('a')
    mouse.rat.vec = mouse.rat.vec[!mouse.rat.vec %in% bad.genes.q]

    #take only pmids and abstract from the query resul
    abstracts = mclapply(seq_along(query.result[[1]]), function(x) {
        query.result[[1]][[x]] %>%
        select(pmid, abstract)
    }, mc.cores = 32) %>% setNames(names(query.result[[1]]))

    #isolate genes from all abstracts
    abstracts.genes.filtered = mclapply(seq_along(abstracts), function(y)
        lapply(1:nrow(abstracts[[y]]), function(x) abstracts[[y]][x,2] %>%
            remove.punc() %>% match.genes(mouse.rat.vec)) %>% unlist() %>% na.omit, mc.cores = 32) %>% setNames(names(abstracts))

   

                                   
    #remove empty gene matches
    abstracts.genes.filtered = abstracts.genes.filtered[names(lapply(abstracts.genes.filtered, length)[lapply(abstracts.genes.filtered, length) > 0])]

    #count the genes for all queried chemicals with genes mentioned in abstracts
    query.result.gene.counts = mclapply(seq_along(abstracts.genes.filtered), function(x) {
        abstracts.genes.filtered[[x]] %>%
            table() %>%
            data.frame() %>%
            setNames(c('gene', 'count')) %>%
            mutate(gene = as.character(gene)) %>%
                            mutate(rat = nh.gene.list$human[match(gene, nh.gene.list$rat)], mouse = nh.gene.list$human[match(gene, nh.gene.list$mouse)]) %>%
                            mutate(gene = ifelse(is.na(rat), gene, rat), gene = ifelse(is.na(mouse), gene, mouse)) %>%
            select(gene, count) %>%
            group_by(gene) %>%
            summarise(count = sum(count)) %>%
            arrange(desc(count))}, mc.cores = 32) %>% setNames(names(abstracts.genes.filtered))

    #calulate the final counts matrixs for all fo the samples
    abstract.gene.count.matrix = query.result.gene.counts %>% reduce(full_join, by = "gene") %>%
        arrange(gene) %>%
        column_to_rownames('gene') %>%
        t() %>%
        as.data.frame() %>%
        replace(is.na(.), 0) %>%
        rownames_to_column('sample') %>%
        select(-sample) %>%
        mutate(chemical = names(query.result.gene.counts)) %>%
        column_to_rownames('chemical')

return(abstract.gene.count.matrix)

}

In [21]:
abstract.gene.count.matrix %>% rownames_to_column('chemicals') %>% write_csv(., path = "output/LINCS-chemical.no.map-abstract-gene-counts-2021.09.17.csv" )

In [23]:
save(abstract.gene.count.matrix, file = "../AbstractR-projects/LINCS/output/gene.counts.matrix-2021.09.17-stress.only.RData")

---
# PMI calculation

In [130]:
#This function reads a ```query_object``` output by the ```query_result``` function. The result is a list of 
#parwise mututal information scores resulting from 

PMI_query_combinations = function(query_result_object, chemical.list.path, search.min = 0, search.max = 100000, num.cores = 1) {

    #Libraries
    library(tidyverse)
    library(parallel)
    library(magrittr)

    #testingpaths
    #  query_result_object = "input/build/pmi.testing.query.object.RData"
    # chemical.list.path = "input/build/pmi.testing.chem.list.csv"
    # search.min = 5
    # search.max = 100000
    #  num.cores = 15

        #read query result
    load(paste0(query_result_object))
    chems.data = suppressMessages(read_csv(paste0(chemical.list.path))) %>%
        mutate(q.chemical = paste0('"',chemicals,'"'))

#take only pmids and abstract from the query result and parse by/ aggregate by SRP/query
search_matrix.i = mclapply(seq_along(query.result[[1]]), function(x) { query.result[[1]][[x]] %>%
        select(pmid) %>% distinct() %>% setNames(c("pmid")) %>% mutate(query = names(query.result[[1]][x]))}, mc.cores = num.cores) %>%
          do.call(rbind, .) %>%
         left_join(query.result[[2]], 'query') %>%
         mutate(search = query) %>%
        separate(col = search, into = c('q.chemical', 'search'), sep = ' AND  ') %>%
        left_join(chems.data, 'q.chemical') %>%
        mutate(search = gsub('\\"','', search)) %>%
        group_split(SRP)

 # aggregate by SRP and chemcial name to remove non-inique PMIDs and remove negative query phrases

search_matrix = mclapply(seq_along(search_matrix.i), function(y){
     search_matrix.i[[y]] %>% ungroup() %>%
     group_by(SRP, chemicals, pmid) %>%
     summarise(dir.tot = direction %>% unique %>% sort %>% paste(collapse = ", ")) %>% #gets rid of negative querried PMIDs
     filter(!grepl(pattern = "neg", x = dir.tot)) %>%
     ungroup() %>%
     group_by(SRP, chemicals) %>%
     summarise(count = pmid %>% unique() %>% length) %>% #takes the unqiue PMID vector length as n unique searches
     ungroup()
}, mc.cores = num.cores) %>%
     do.call(rbind, .) %>%
     pivot_wider(names_from = SRP, values_from = count, values_fill = 0) %>%
     filter(!is.na(chemicals)) %>%
     column_to_rownames("chemicals") %>%
     mutate(rsum = rowSums(across(where(is.numeric))))

     
     



    #filter based on search returns
    search_matrix.wide.f = search_matrix %>%
        filter(rsum >= search.min & rsum <= search.max) %>%
        select(-rsum)


    #build a result matrix
    search_PMI = matrix(nrow = nrow(search_matrix.wide.f), ncol = ncol(search_matrix.wide.f)) %>%
        magrittr::set_rownames(rownames(search_matrix.wide.f)) %>%
        magrittr::set_colnames(colnames(search_matrix.wide.f))

    #Calculate PMI
    for (i in 1:nrow(search_matrix.wide.f)) {
        for (j in 1:ncol(search_matrix.wide.f)){
        search_PMI[i, j] <- (log2(sum(search_matrix.wide.f[i,])/sum(search_matrix.wide.f))
            + log2(sum(search_matrix.wide.f[,j])/sum(search_matrix.wide.f))
            -log2(search_matrix.wide.f[i,j]/sum(search_matrix.wide.f))) *-1
        }
    }
  

    #clean up infs
    search_PMI = search_PMI %>%
    replace(is.infinite(.), 0)

  
    return(search_PMI)
    
}

In [ ]:
PMI_query_combinations()

---
# select.highest


---
# search hits table

In [1]:
#This function reads a ```query_object``` output by the ```query_result``` function. The result is a list of 
#parwise mututal information scores resulting from 

search_query_combinations = function(query_result_object, chemical.list.path, search.min = 0, search.max = 100000) {

    #Libraries
    library(tidyverse)
    library(parallel)
    library(magrittr)

    #testingpaths
#     query_result_object = "../AbstractR-projects/LINCS/output/query.result_LINCS_2021-07-29.RData"
#     chemical.list.path = "input/chemical-list-LINCS-2021.07.23.csv"
#     search.min = 5
#     search.max = 100000
    

    #read query result
    load(paste0(query_result_object))
    chems.data = suppressMessages(read_csv(paste0(chemical.list.path))) %>%
        mutate(q.chemical = paste0('"',chemicals,'"'))

#take only pmids and abstract from the query result and calc number results
search_matrix = mclapply(seq_along(query.result[[1]]), function(x) { query.result[[1]][[x]] %>%
        pull(pmid) %>% unique() %>% length() }, mc.cores = 10) %>% setNames(names(query.result[[1]])) %>% unlist() %>%
        as.data.frame() %>%
        setNames('search_results') %>%
        rownames_to_column('query') %>%
        left_join(query.result[[2]], 'query') %>%
        mutate(search = query) %>%
        separate(col = search, into = c('q.chemical', 'search'), sep = ' AND  ') %>%
        left_join(chems.data, 'q.chemical') %>%
        mutate(search = gsub('\\"','', search)) %>%
        select(chemicals, SRP, search, direction, search_results)

    #make a wide form matrix
    search_matrix.wide = search_matrix %>%
        select(chemicals, search, search_results) %>%
        pivot_wider(id_cols = chemicals, names_from = search, values_from = search_results) %>%
        replace(is.na(.), 0) %>%
        mutate(rsum = rowSums(across(where(is.numeric))))

    #filter based on search returns
    search_matrix.wide.f = search_matrix.wide %>%
        filter(rsum >= search.min & rsum <= search.max) %>%
        select(-rsum) %>%
        column_to_rownames('chemicals')



  
    return(search_matrix.wide.f)
    
}

In [124]:
#This function takes a numeric matrix resulting from PMI, GSEA (connectivity), or some other metric and 
#' returns that same matrix with two additional columns: the name of the highest scoring column for each row and
#' the corresponding score

select.highest = function(score.matrix) {

    #Libraries
    suppressMessages(library(tidyverse))
    
    #testingpaths
    #score.matrix = score.matrix
    

    #maxcolumn and value
    score.matrix.highest = data.frame(max_column = colnames(score.matrix)[apply(score.matrix, 1, which.max)], stringsAsFactors = FALSE) %>%
    cbind(score.matrix %>% apply(1, max) %>% as.data.frame() %>% setNames("max_value") %>%
    rownames_to_column(var = "row"), .) %>%
    left_join(score.matrix %>% as.data.frame() %>% rownames_to_column('row'), by = 'row') %>%
    column_to_rownames('row')

return(score.matrix.highest)
}

---
# WORK FLOW
---

In [2]:
suppressMessages(R.utils::sourceDirectory("lib/", modifiedOnly = FALSE, recursive = TRUE))

In [ ]:
query.result = query_result(chemical.list.path = "input/chemical-list-LINCS-2021.07.23.csv", query.map.path = "input/query.map.csv", limit_input_to = 10)

In [6]:
load("input/build/pmi.testing.query.object.RData")

In [4]:
query.result[[3]]

SRP,phrase,direction,phrase.q
<chr>,<chr>,<chr>,<chr>
DDR,dna damage,pos,"""dna damage"""
UPR,er stress,pos,"""er stress"""
UPR,unfolded protein response,pos,"""unfolded protein response"""
HSR,heat shock,pos,"""heat shock"""
HPX,hypoxia,pos,"""hypoxia"""
MTL,metal stress,pos,"""metal stress"""
MTL,chelator,neg,"""chelator"""
OSR,oxidative stress,pos,"""oxidative stress"""
OSR,antioxidant,neg,"""antioxidant"""


In [35]:
#save(query.result, file = "input/pmi.testing.query.object.RData")

In [1]:
count.matrix = abstract_gene_counts(query_result_object = "input/build/pmi.testing.query.object.RData")

ERROR: Error in abstract_gene_counts(query_result_object = "input/build/pmi.testing.query.object.RData"): could not find function "abstract_gene_counts"


In [9]:
count.matrix

,AATF,ABCA1,ABCC1,ABCC2,ABCC3,ABCC5,ABCC6,ABCC8,ABCG2,ABCG4,⋯,XPC,XRCC1,XRCC3,XRCC5,XRCC6,YAP1,YWHAZ,ZAP70,ZFC3H1,ZIC3
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"""(+)-3-(1-propyl-piperidin-3-yl)-phenol"" AND ""dna damage""",5,3,1,0,0,0,0,0,0,1,⋯,3,4,4,1,2,0,0,1,0,0
"""1,2,3,4-tetrahydroisoquinoline"" AND ""dna damage""",0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
"""(+)-3-(1-propyl-piperidin-3-yl)-phenol"" AND ""er stress""",0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
"""(+)-3-(1-propyl-piperidin-3-yl)-phenol"" AND ""unfolded protein response""",0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
"""(+)-3-(1-propyl-piperidin-3-yl)-phenol"" AND ""heat shock""",0,0,2,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,2,0,0,0
"""1,2,3,4-tetrahydroisoquinoline"" AND ""heat shock""",0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
"""(+)-3-(1-propyl-piperidin-3-yl)-phenol"" AND ""hypoxia""",0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,1,0
"""1,2,3,4-tetrahydroisoquinoline"" AND ""hypoxia""",0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
"""(+)-3-(1-propyl-piperidin-3-yl)-phenol"" AND ""metal stress""",0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [98]:
PMI_matrix = PMI_query_combinations(query_result_object = "input/build/pmi.testing.query.object.RData", chemical.list.path = "input/chemical-list-LINCS-2021.07.23.csv", search.min = 0, search.max = 100000)

In [99]:
PMI_matrix

,dna damage,er stress,unfolded protein response,heat shock,hypoxia,metal stress,chelator,oxidative stress,antioxidant
(+)-3-(1-propyl-piperidin-3-yl)-phenol,-0.003703959,0.001878590,-0.01570382,-0.0004072132,0.001498881,0.007755157,-0.001852171,-0.002881479,0.003433551
1-benzylimidazole,1.257411714,0.000000000,0.00000000,0.0000000000,0.000000000,0.000000000,2.741013500,0.000000000,0.854120978
"1,2-dichlorobenzene",0.878900091,0.000000000,0.00000000,0.7130390220,0.000000000,0.000000000,0.000000000,1.054256825,-2.109353146
"1,2,3,4-tetrahydroisoquinoline",0.455957394,0.009868715,1.99818691,0.0677039033,0.100009770,0.000000000,0.132204257,0.354473922,-0.880219147
1-monopalmitin,0.000000000,0.000000000,0.00000000,0.0000000000,0.000000000,0.000000000,0.000000000,1.584771542,0.000000000


In [125]:
PMI_matrix_best = select.highest(score.matrix = PMI_matrix)

In [126]:
PMI_matrix_best

,max_value,max_column,dna damage,er stress,unfolded protein response,heat shock,hypoxia,metal stress,chelator,oxidative stress,antioxidant
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(+)-3-(1-propyl-piperidin-3-yl)-phenol,0.007755157,metal stress,-0.003703959,0.001878590,-0.01570382,-0.0004072132,0.001498881,0.007755157,-0.001852171,-0.002881479,0.003433551
1-benzylimidazole,2.741013500,chelator,1.257411714,0.000000000,0.00000000,0.0000000000,0.000000000,0.000000000,2.741013500,0.000000000,0.854120978
"1,2-dichlorobenzene",1.054256825,oxidative stress,0.878900091,0.000000000,0.00000000,0.7130390220,0.000000000,0.000000000,0.000000000,1.054256825,-2.109353146
"1,2,3,4-tetrahydroisoquinoline",1.998186909,unfolded protein response,0.455957394,0.009868715,1.99818691,0.0677039033,0.100009770,0.000000000,0.132204257,0.354473922,-0.880219147
1-monopalmitin,1.584771542,oxidative stress,0.000000000,0.000000000,0.00000000,0.0000000000,0.000000000,0.000000000,0.000000000,1.584771542,0.000000000


accuracy rank

In [ ]:
#this function takes a matrix of measures for a sample set, along side a validated columns, and measures the weighted accuracy of each assignment
function(score.matrix, val.col, measures.cols){
     matrix = score.matrix

     
}

---
STRACTH and LEARNING